# CNN from scratch

This notebook aims to solve Lung CT segmentation using Convolutional Neural Network.

First, some imports.

In [ ]:
import numpy as np
from utils.data_utils import read_data
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose
from keras.layers import Cropping2D, Concatenate
from keras.models import Model

Then, read the data.

In [ ]:
x_train, y_train = read_data()

# check data size
print(x_train.shape, y_train.shape)

# U-Net

First, we use the popular U-Net structure.

In [ ]:
# build the model
inputs = Input(shape=(,))
x = Conv2D(64, (3, 3), activation='relu')(inputs)